In [1]:
import pandas as pd
import numpy as np 
import requests
import json
import base64
import datetime
from urllib.parse import urlencode
import time

import urllib.error
import urllib.request

from tenacity import retry, retry_if_exception, stop_after_attempt, wait_fixed
from tenacity.wait import wait_base


In [2]:
client_id = ''
client_secret = ''

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}?ids={lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    


In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [5]:
def get_data(query, api, num):
    chunk = api.get_resource(query, 'audio-features', 'v1')
    json_object = json.dumps(chunk, indent = 4)

    #with open(f'C:/Users/Administrator/data/spotify_scrape/songset{num}.json', 'w') as outfile:
        #outfile.write(json_object)

In [6]:
ids_list = np.array(pd.read_csv('C:/Users/Administrator/data/170k_songs.csv')['track_uri'])
splitted_first_half = (np.array_split(ids_list[:170000], 1700))
splitted_second_half = list(ids_list[170000:])
bruh = list(splitted_first_half)
bruh.append(splitted_second_half)

In [7]:
def perform():
    tracker = 0
    for i in bruh:
        qry = ','.join(i)
        complete = False
        while complete != True:
            try:
                get_data(qry, spotify, tracker)
                complete = True
            except :
                print('http error')
                time.sleep(60)
                
        tracker += 1
        if tracker % 1000 == 0:
            print(tracker)
        time.sleep(10)


In [8]:
#perform()

1000
http error
